<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/dnn_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 

<!--NAVIGATION-->
 < [Climate Change - CMIP5](cmip5.ipynb) | [Contents](index.ipynb) | [Crop Area Prediction Using a Convolutional Neural Network](CNN_demo.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/dnn_demo.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<center>
<h1>Google Earth Engine with Python </h1>
<h2> Integrating Earth Engine with Tensorflow I - DNN </h2>
</center>

This notebook has been inspired by the [Chris Brown & Nick Clinton EarthEngine + Tensorflow presentation](https://www.youtube.com/watch?v=w-1xfF0IaeU). It shows the step by step how to integrate Google Earth Engine and TensorFlow 2.0 in the same pipeline (EE->Tensorflow->EE).

```
FILE: dnn_demo.ipynb
AUTHOR: Cesar Aybar
EMAIL: csaybar@gmail.com
ORGANIZATION: n/a 
MODIFIED BY: n/a
CREATION DATE: 23 May. 2019
LAST MOD DATE: N/A
DEPENDENCIES: earthengine-api, os, pandas, urllib, plotly, numpy, folium
PURPOSE: Introduction to tensorflow & GEE integration
```

<center>
<img src="https://raw.githubusercontent.com/csaybar/EEwPython/master/images/colab_ee_integration.png">
</center>  

## Topics

1. Create a training/testing dataset (in a TFRecord format) using Earth Engine.
2.  Create functions for parse data (TFRecord -> tf.data.Dataset).
3.   Training and Test a simple Vanilla Deep Neural Network using tensorflow 2.0.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Upload your results to Earth Engine (asset).




## 1. Introduction

Deep learning has dramatically improved the state-of-the-art in various science domains. For remote sensing, its potential has not been thoroughly explored. This could be related to the problematic incorporation of spectral & spatial features into a regular deep learning classification scheme or the huge pre-processing that satellite images could need it. Hence, this post aims to teach you how to create a painless deep learning workflow integrating [Google Earth engine](https://earthengine.google.com/) for acquiring spectral & spatial data and [tensorflow](https://www.tensorflow.org/?hl=en) for train and test the model and make predictions.


## 2. What is Google Earth Engine (GEE)?

In a nutshell, is a platform that combines a multi-petabyte [catalog of satellite imagery](https://developers.google.com/earth-engine/datasets/) with planetary-scale analysis capabilities. There are several ways to interact with GEE:

  - [Explorer](https://explorer.earthengine.google.com/)
  - [Code Editor](https://code.earthengine.google.com/)  
  - [Javascript wrapper library](https://github.com/google/earthengine-api/tree/master/javascript)
  - [**Python wrapper library**](https://github.com/google/earthengine-api/tree/master/python)

In my opinion, the **Python wrapper library** (used in this post) is the best choice to interact with GEE for the following reasons:

- Easy to share code.
- Easy transition to a web application.
- Possibility to integrate with  ML/DL frameworks.
- Many plotting options (folium, plotly, matplotlib, seaborn ,etc.).

One more thing!, it's possible to run the Earth Engine Python API in a cloud environment for free. See the introduction of the [EEcourse](https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/1_Introduction.ipynb) for more details.

## 3.  Deep Neural Network (DNN)

<center>
<image src="https://www.electronicdesign.com/sites/electronicdesign.com/files/MachineLearning_WTD_Fig3.png">
</center>
  
A DNN is simply a neural network with more than two layers. The main steps for building a DNN are:

1. Initialize the model's parameters.
2. Loop:
    - Calculate current loss (forward propagation) :    
    - Calculate current gradient (backward propagation)
    - Update parameters (gradient descent)

The second step could be  a little intimidating, but don't worry about it!. **tf.keras**, the TensorFlow's high-level API, only need that you define the forward propagation correctly and all the steps further down will make automatically. This post does not intend to introduce the algorithm, check out this [repo](https://github.com/csaybar/DLcoursera) for a from zero (numpy) implementation.

## 4. Crop Area estimation in Camana Valley (DEMO)

Agriculture is part of the backbone Peruvian economy, contributing about 7.6% of the Gross Domestic Product (GDP), being more critical in rural areas where the contribution of GDP increase until 50%. In terms of people, this activity act for the primary source of income for 2.3 million families, representing 34% of Peruvian households. Despite agriculture importance in Peruvian family lives, today no exist a cropping system either at a national or regional scale that monitoring the extension, state, or crop type. Considering this problematic, in this section **you** will create a straightforward methodology to **predict the crop area** in Camana (Arequipa) Valley using a Deep Neural Network.

<center>
<img src='https://st.depositphotos.com/1171712/3974/i/950/depositphotos_39741899-stock-photo-camana-valley.jpg'>
</center>


In [0]:
#@title Mapdisplay: Display ee.Features and ee.Images using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### 4.1. Installing

Before coding do not forget install and load the following packages and remenber that you can communicate with the bash console prepending an ! to the code. 




In [0]:
# This tensorflow version is necessary just for tensorboard support.
# developer version, on future it maybe breaks. 
!pip install tf-nightly-2.0-preview==2.0.0.dev20190606 

!pip install earthengine-api==0.1.175 #earthengine API
# Load the TensorBoard notebook extension
%load_ext tensorboard

### 4.2. Authentification

This tutorial needs interacting with some Google services. For accomplish this task, it's necessary to authenticate (as yourself). The code below shows you how to do it.

#### Google Cloud

Google Cloud Storage bucket will serve as a bridge between GEE and Colab.


In [0]:
from google.colab import auth
auth.authenticate_user()

#### Google Earth Engine

In [0]:
!earthengine authenticate

### 4.3. Initialize and testing the software setup



In [7]:
# Earth Engine Python API
import ee 
ee.Initialize()

import tensorflow as tf
print('Tensorflow version: ' + tf.__version__)

import folium
print('Folium version: ' + folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

Tensorflow version: 2.0.0-dev20190606
Folium version: 0.8.3


### 4.4. Prepare the Dataset

Firstly, we define our prediction area (Camana Valley) and passing to GEE. For moving a vector to GEE, you will use the `ee.Geometry.*` module. The [GeoJSON](https://geojson.org/) spec describes in detail the type of geometries supported by GEE, including `Point` (a list of coordinates in some projection), `LineString` (a list of points), `LinearRing` (a closed LineString), and `Polygon` (a list of LinearRings where the first is a shell and subsequent rings are holes). GEE also supports **MultiPoint**, **MultiLineString**, and **MultiPolygon**. The [GeoJSON](https://geojson.org/) GeometryCollection is also supported, although it has the name **MultiGeometry** within GEE.

In [10]:
# 2.4.1 Prediction Area
xmin,ymin,xmax,ymax = [-72.778645, -16.621663, -72.66865, -16.57553]


# Passing a rectangle (prediction area) to Earth Engine
Camana_valley = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])
center = Camana_valley.centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'Camana Valley':Camana_valley.getInfo()},zoom_start=12)

Next, you will read and create a visualization of the train/test dataset. I've already generated some points with the label agriculture/non-agriculture.

- Train dataset (550 points):
  - 275 labeled as "agriculture"
  - 275 labeled as "non agriculture" 
  
- Test dataset (100 points):
  - 50  labeled as "agriculture"
  - 50  labeled as "non agriculture"

In [11]:
# 2.4.2 Importing the train/test dataset
train_agriculture = ee.FeatureCollection('users/csaybar/DLdemos/train_set') 
test_agriculture = ee.FeatureCollection('users/csaybar/DLdemos/test_set')

# Display the train/test dataset
db_crop = train_agriculture.merge(test_agriculture)
center = db_crop.geometry().centroid().getInfo()['coordinates']
center.reverse()

dicc = {'train': train_agriculture.draw(**{'color': 'FF0000', 'strokeWidth': 5}).getMapId(),
        'test' : test_agriculture.draw(**{'color': '0000FF', 'strokeWidth': 5}).getMapId(),
        'CamanaValley':Camana_valley.getInfo()
       }

Mapdisplay(center,dicc,zoom_start=8)

In this part, you will obtain the input data for mapping the **Camana crop area** using  [Landsat 8 OLI/TIRS (L8)](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR). GEE provides L8 images with radiometric and geometry correction. Additionally, **cloud mask information** is provided by means of the bit image `pixel_qa`. The following function allows putting NA to the TOA reflectance values of clouds.


In [0]:
def maskS2clouds(img):
  '''  
  Function to mask clouds based on the pixel_qa band of Landsat 8 SR data. See:
  https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR
  
  Params:
  -------
  - img: image input Landsat 8 SR image
  
  Return:
  -------
  cloudmasked Landsat 8 image
  '''
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = img.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
           .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return img.updateMask(mask)     

Now you will filter and reduce the entire Landsat-8 dataset, considering the following:

1. Select just bands **[R, G, B, NIR]**.

2. Filter considering the cloud pixel percentage by scene (< 20%).

3. Filter considering a date (we just selecting 1 years)

4. Apply **mask2cloud** to each image.

5. Get the median of the ImageCollection.

6. Clip the image considering study area.

**NOTE:** To apply a function on all the elements of specified **`ImageCollection`** or **`FeatureCollection`**, you can use the **`map()`** function. 

In [0]:
# 2.4.3 Prepare the satellite image (Landsat-8)
RGB_bands = ['B4','B3','B2'] #RGB
NDVI_bands = ['B5','B4'] #NIR

l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
               .filterBounds(db_crop)\
               .filterDate('2018-01-01', '2018-12-31')\
               .filter(ee.Filter.lt('CLOUD_COVER', 20))\
               .map(maskS2clouds)\
               .median()\
               .multiply(0.0001)

l8_ndvi = l8.normalizedDifference(NDVI_bands).rename(['NDVI'])
l8_rgb = l8.select(RGB_bands).rename(['R','G','B']) 
l8 = l8_rgb.addBands(l8_ndvi)

In [14]:
from collections import OrderedDict
# Create a visualization with folium
visParams_l8 = {    
  'bands': ['R', 'G', 'B'],
  'min': 0,
  'max': 0.5,
  'gamma': 1.4,
}

l8Mapid = l8.getMapId(visParams_l8)
dicc['Landsat8'] = l8Mapid

# Changing the order of the dictionary
key_order = ['Landsat8','CamanaValley','train','test']
dicc = OrderedDict((k, dicc[k]) for k in key_order)

Mapdisplay(center,dicc,zoom_start=8)

For the collocation of the train/test datasets to each L8 grid cell value you will use the `ee.Image.sampleRegions` function. Note that geometries will be snapped to pixel centers (nearest center approximation).




In [0]:
# 2.4.3 Extract pixels values considering train/test dataset
train_db = l8.sampleRegions(collection=train_agriculture, properties=['class'], scale=30)
test_db = l8.sampleRegions(collection=test_agriculture, properties=['class'], scale=30)

Unfortunately, you cannot use Tensorflow directly in Earth Engine. To overcome this problem, you will save the train/test dataset into a Google **Cloud Storage Bucket (GCS)** (you could use Google Drive instead), since both GEE and GCS and Tensorflow can access to them. More details about how to export data in GEE you can find them in the next [link](10_Export.ipynb) or into the [Official Exporting data guide](https://developers.google.com/earth-engine/exporting).

In [16]:
# 2.4.4 Save results in Google Cloud Storage
outputBucket = 'bag_csaybar' # Replace with your Cloud Storage bucket 

# Make sure the bucket exists.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + outputBucket) 
    else 'Output Cloud Storage bucket does not exist.')

trainFilePrefix = 'TrainingAgriculture_'
testFilePrefix = 'TestAgriculture_'

# Create the tasks for passing of GEE to Google storage
trainingTask = ee.batch.Export.table.toCloudStorage(
  collection=train_db,
  description='Training Export',
  fileNamePrefix=trainFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord')

testingTask = ee.batch.Export.table.toCloudStorage(
  collection=test_db,
  description='Testing Export',
  fileNamePrefix=testFilePrefix,
  bucket=outputBucket,
  fileFormat='TFRecord')

trainingTask.start()
testingTask.start()

Found Cloud Storage bucket.


In [17]:
# Monitor task progress
# Code Extracted here:
# https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb
import time 
while trainingTask.active():
  print('Polling for task (id: {}).'.format(trainingTask.id))
  time.sleep(5)
while testingTask.active():
  print('Polling for task (id: {}).'.format(testingTask.id))
  time.sleep(5)

print('Done!')

Polling for task (id: VPBH2OPHEXI6EFSCSJDT2RHT).
Polling for task (id: VPBH2OPHEXI6EFSCSJDT2RHT).
Polling for task (id: VPBH2OPHEXI6EFSCSJDT2RHT).
Polling for task (id: VPBH2OPHEXI6EFSCSJDT2RHT).
Polling for task (id: VPBH2OPHEXI6EFSCSJDT2RHT).
Done!


### 4.5. Creating a tf.data.Dataset from a TFRecord file

Read data from the TFRecord file into a tf.data.Dataset. Pre-process the dataset to get it into a suitable format for input to the DNN model. 
For getting more details about  `tf.data.Dataset`see the next [TFdoc](https://www.tensorflow.org/guide/premade_estimators#create_input_functions).








In [0]:
# Fullname train/test db
fileNameSuffix = 'ee_export.tfrecord.gz'
trainFilePath = 'gs://' + outputBucket + '/' + trainFilePrefix + fileNameSuffix
testFilePath = 'gs://' + outputBucket + '/' + testFilePrefix + fileNameSuffix

In [0]:
def input_fn(fileNames, numEpochs=None, shuffle=True, batchSize=16):
  # Read `TFRecordDatasets` 
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')

  # Names of the features 
  feature_columns = {
    'R': tf.io.FixedLenFeature([], dtype=tf.float32),  
    'G': tf.io.FixedLenFeature([], dtype=tf.float32),  
    'B': tf.io.FixedLenFeature([], dtype=tf.float32),    
    'NDVI': tf.io.FixedLenFeature([], dtype=tf.float32),    
    'class': tf.io.FixedLenFeature([], dtype=tf.float32)
  }
  
  # Make a parsing function
  def parse(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_columns)    
    # Separate the class labels from the training features
    labels = parsed_features.pop('class')
    return parsed_features, tf.cast(labels, tf.int32)
  
  # Map the function over the dataset
  dataset = dataset.map(parse, num_parallel_calls=5)
  if shuffle:
    dataset = dataset.shuffle(buffer_size = batchSize * 10)
  dataset = dataset.batch(batchSize)
  dataset = dataset.repeat(numEpochs)
  
  return dataset

In [0]:
train_dba = input_fn(trainFilePath,100,True,32)
test_dba = input_fn(testFilePath, numEpochs=1, batchSize=1, shuffle=False)

### 4.6. Create a DNN model with keras

Here you will create a deep neural network model with:
- 2 layers (10x10 nodes).
- 1 dropout layer.
- 1 output layer.

Additionally, Early Stopping, Tensorboard, and best model callback were added. A callback is a set of functions to be applied at given stages of the training procedure. You can found more details [here](https://keras.io/callbacks/).


In [22]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import feature_column
import os
import datetime


bands = ['R','G','B','NDVI']

# Create a dense `Tensor` based on given `feature_columns`.
feature_columns = [feature_column.numeric_column(x) for x in bands]
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

# Initialize the DNN model
he_init = tf.keras.initializers.he_uniform(seed=None)

# Define the layers in the model.
model = tf.keras.Sequential([
  feature_layer,  
  layers.Dense(10, activation='relu',kernel_initializer=he_init),
  tf.keras.layers.Dropout(0.2),
  layers.Dense(10, activation='relu',kernel_initializer=he_init),
  layers.Dense(1, activation='sigmoid',kernel_initializer=he_init)
])

# Callbacks time
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
es = EarlyStopping(monitor='val_loss', patience=10)
mcp = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)


# Compile the model with the specified loss function.
model.compile(optimizer=keras.optimizers.SGD(momentum=0.01, nesterov=True),
              loss='binary_crossentropy',              
              metrics=['accuracy'])

# Fit the model to the training data.
model.fit(x=train_dba,
          epochs=15,
          steps_per_epoch=100,
          callbacks=[tensorboard_callback,es,mcp],
          validation_data=test_dba)

Epoch 1/15


W0608 00:46:57.023546 139817096210304 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1251: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100/100 [==============================] - 3s 25ms/step - loss: 0.6591 - accuracy: 0.6542 - val_loss: 0.6196 - val_accuracy: 0.9500
Epoch 2/15
100/100 [==============================] - 0s 5ms/step - loss: 0.6167 - accuracy: 0.7760 - val_loss: 0.5709 - val_accuracy: 0.9400
Epoch 3/15
100/100 [==============================] - 1s 5ms/step - loss: 0.5851 - accuracy: 0.8135 - val_loss: 0.5285 - val_accuracy: 0.9300
Epoch 4/15
100/100 [==============================] - 0s 5ms/step - loss: 0.5535 - accuracy: 0.8224 - val_loss: 0.4882 - val_accuracy: 0.9400
Epoch 5/15
100/100 [==============================] - 0s 5ms/step - loss: 0.5248 - accuracy: 0.8280 - val_loss: 0.4500 - val_accuracy: 0.9500
Epoch 6/15
100/100 [==============================] - 0s 5ms/step - loss: 0.5039 - accuracy: 0.8268 - val_loss: 0.4166 - val_accuracy: 0.9500
Epoch 7/15
100/100 [==============================] - 0s 5ms/step - loss: 0.4814 - accuracy: 0.8071 - val_loss: 0.3875 - val_accuracy: 0.9500
Epoch 8/15
100/1

In [0]:
%tensorboard --logdir logs
#!kill 607

In [24]:
model.load_weights("best_model.h5")
model.evaluate(x = test_dba)

    100/Unknown - 1s 5ms/step - loss: 0.2560 - accuracy: 0.9300

[0.25602747334167364, 0.93]

### 4.7. Prediction

You will prepare the L8 imagery, likewise, you made it for the train/test dataset. 

In [0]:
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
               .filterBounds(Camana_valley)\
               .filterDate('2018-01-01', '2018-12-31')\
               .filter(ee.Filter.lt('CLOUD_COVER', 20))\
               .map(maskS2clouds)\
               .median()\
               .multiply(0.0001)\
               .clip(Camana_valley)

l8_ndvi = l8.normalizedDifference(NDVI_bands).rename(['NDVI'])
l8_rgb = l8.select(RGB_bands).rename(['R','G','B']) 
l8 = l8_rgb.addBands(l8_ndvi)

For export the results to the Google Cloud Storage, it's preferred defines the following `formatOptions` parameters to save memory:

- **patchDimensions**:	Patch dimensions tiled over the export area, covering every pixel in the bounding box exactly once (except when the patch dimensions do not evenly divide the bounding box in which case the lower and right sides are trimmed).

- **compressed**: If true, compresses the .tfrecord files with gzip and appends the ".gz" suffix	

See all the paramerters [here](https://developers.google.com/earth-engine/exporting#configuration-parameters).

In [0]:
imageFilePrefix = 'CamanaValleyCrop'

# Specify patch and file dimensions.
imageExportFormatOptions = {
  'patchDimensions': [64, 64],  
  'maxFileSize': 100000000,    
  'compressed': True
}

# Setup the task.
imageTask = ee.batch.Export.image.toCloudStorage(
  image=l8,
  description='Image Export',
  fileNamePrefix=imageFilePrefix,
  bucket=outputBucket,
  scale=30,
  fileFormat='TFRecord',
  region=Camana_valley.getInfo()['coordinates'],
  formatOptions=imageExportFormatOptions,
)

imageTask.start()

In [27]:
import time 
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(5)

NameError: ignored

Now it's time to classify the image that was exported from GEE to GCS using Tensorflow. If the exported image is large (it not your case), it will be split into multiple TFRecord files in its destination folder. There will also be a JSON sidecar file called **"the mixer"** that describes the format and georeferencing of the image. Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

In [28]:
filesList = !gsutil ls 'gs://'{outputBucket}
exportFilesList = [s for s in filesList if imageFilePrefix in s]

# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.tfrecord.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f

# Make sure the files are in the right order.
print(jsonFile)

gs://bag_csaybar/CamanaValleyCrop.json


The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file. Read the mixer to get some information needed for prediction.

In [29]:
import json
from pprint import pprint 

# Load the contents of the mixer file to a JSON object.
jsonText = !gsutil cat {jsonFile}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

{'patchDimensions': [64, 64],
 'patchesPerRow': 6,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -72.77889920503969,
                                            0.0,
                                            -0.00026949458523585647,
                                            -16.57526446493135]},
                'crs': 'EPSG:4326'},
 'totalPatches': 12}


The next function  is slightly different from the to the `input_fn` (see Section 4.5). Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.  Once the `predict_input_fn` is defined, that can handle the shape of the image data, all you need to do is feed it directly to the trained model to make predictions.


In [0]:
def predict_input_fn(fileNames,patch,bands):

  # You have to know the following from your export.
  PATCH_WIDTH, PATCH_HEIGHT = patch
  PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]

  # Note that the tensors are in the shape of a patch, one patch for each band.
  imageColumns = [
    tf.io.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) 
      for k in bands
  ]

  featuresDict = dict(zip(bands, imageColumns))
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')
  
  # Make a parsing function
  def parse_image(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
    return parsed_features

  dataset = dataset.map(parse_image, num_parallel_calls=4)
  
  # Break our long tensors into many littler ones
  #https://stackoverflow.com/questions/50530806/using-flat-map-in-tensorflows-dataset-api
  dataset = dataset.flat_map(lambda features: tf.data.Dataset.from_tensor_slices(features))

  # Read in batches corresponding to patch size.
  dataset = dataset.batch(PATCH_WIDTH * PATCH_HEIGHT)

  return dataset

In [0]:
predict_db = predict_input_fn(fileNames=imageFilesList,patch=[64,64],bands=['R', 'G', 'B', 'NDVI'])
predictions = model.predict(predict_db)

Now that there's a `np.array` of probabilities in "predictions", it's time to write them back into a file. You will write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list and write the probabilities in patches. Specifically, we need to write the pixels into the file as patches in the same order they came out. The records are written as serialized tf.train.Example protos. This might take a while.

In [0]:
# Instantiate the writer.
PATCH_WIDTH , PATCH_HEIGHT = [64,64]
outputImageFile = 'gs://' + outputBucket + '/CamanaValleyCrop.TFRecord'
writer = tf.io.TFRecordWriter(outputImageFile)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = []
curPatch = 1
for  prediction in predictions:
  patch.append(prediction)
  
  if (len(patch) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + '...')    
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'crop_prob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch))
        }
      )
    )
    
    writer.write(example.SerializeToString())
    patch = []
    curPatch += 1 

writer.close()

### 4.8 Upload the classifications to an Earth Engine asset

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket. Use the gsutil command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.




In [33]:
!gsutil ls -l {outputImageFile}

    197172  2019-06-08T00:49:17Z  gs://bag_csaybar/CamanaValleyCrop.TFRecord
TOTAL: 1 objects, 197172 bytes (192.55 KiB)


Upload the image to Earth Engine directly from the Cloud Storage bucket with the [earthengine command](https://developers.google.com/earth-engine/command_line#upload). Provide both the image TFRecord file and the JSON file as arguments to earthengine upload.



In [34]:
# REPLACE WITH YOUR USERNAME:
USER_NAME = 'csaybar'
outputAssetID = 'users/' + USER_NAME + '/CamanaCrop'
print('Writing to ' + outputAssetID)

Writing to users/csaybar/CamanaCrop


In [35]:
# Start the upload. It step might take a while.
!earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

Started upload task with ID: RNBAHAB727R546JYIHARCJUM


Display the Results using Folium!

In [36]:
ProbsImage = ee.Image(outputAssetID)
predictionsImage = ee.Image(outputAssetID).gte(0.505)
dicc = {'CropProbability':ProbsImage.getMapId(),
        'Crop':predictionsImage.getMapId()}

center = Camana_valley.centroid().getInfo()['coordinates']
center.reverse()

Mapdisplay(center=center,dicc=dicc,zoom_start=13)

### That's all for this time!, the next post is about semantic segmentation and Earth Engine.